### Installing & Importing all the necessary packages

Update system packages, install `libgl1`, and install the `openpyxl` library for handling Excel files

- **`albumentations`**: Library for image augmentation to enhance training data diversity.
- **`sweetviz`**: Generates high-density visualizations of pandas DataFrames for quick data analysis.
- **`grad-cam`**: Visualizes important image regions for CNN predictions using Grad-CAM.
- **`lime`**: Provides local explanations for machine learning model predictions.
- **`pandas_profiling`**: Creates detailed reports of pandas DataFrames for exploratory data analysis.
- **`shap`**: Explains model predictions by attributing feature contributions using Shapley values.
- **`Keras-Preprocessing`**: Offers utilities for preprocessing data, including image and text transformations.

In [ ]:
from IPython.display import clear_output

!sudo apt-get update && apt-get install libgl1 -y
!pip install openpyxl -q
!pip install albumentations sweetviz grad-cam lime pandas_profiling shap Keras-Preprocessing -q

clear_output()

In [ ]:
# Standard Libraries
import os
import math
import shutil
import warnings
from typing import Dict, Optional

# Data Handling
import pandas as pd
import numpy as np
import cv2
from PIL import Image

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

# Skimage
from skimage.segmentation import slic, mark_boundaries

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet169, NASNetMobile, EfficientNetB7, ConvNeXtBase, VGG19, Xception
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Keras (Standalone)
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop, Nadam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, classification_report,
                             confusion_matrix, roc_curve, roc_auc_score, matthews_corrcoef, hamming_loss, f1_score, precision_recall_curve)

# Lime and Shap
from lime import lime_image
import shap

# PyTorch Grad-CAM
from pytorch_grad_cam import GradCAM

# Sweetviz (for EDA)
import sweetviz

# IPython for display
from IPython.display import Image, display

# Magic commands (for Jupyter Notebooks)
%matplotlib inline
%load_ext tensorboard
%reload_ext tensorboard

### Data Loading and Cleaning

Load class labels from an Excel file, binary test labels from a CSV, clean the class labels DataFrame by removing empty rows and columns, and display the cleaned data.

In [ ]:
la1=pd.read_excel('/workspace/anushka saini/train_val/multilabelpcos.xlsx')
la1

In [ ]:
la2 = pd.read_csv('/workspace/anushka saini/test/test_label_multi.csv')
la2

In [ ]:
df = la1.dropna(how='all').dropna(how='all', axis=1)
print(df)

### Train-Validate Split

Split the DataFrame into training and validation sets based on specified percentages, using optional random seed for reproducibility.

In [ ]:
def train_validate_split(df, train_percent=.8, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    # test = df.iloc[perm[validate_end:]]
    return train, validate

In [ ]:
train, validate = train_validate_split(df)
train

In [ ]:
validate

### Image and Label Preparation

Map labels, load and preprocess images for training, validation, and testing, and display shapes of image arrays and labels.

In [ ]:
train_labels = train[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
train_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in train['ImagePath']] #to change when using GPU. similarly for validate and test
train_images = []
for train_image_path in train_image_paths:
    image = load_img(train_image_path, target_size=(300, 300)) # change this when you run on GPU. 320X320 rahegi
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    train_images.append(image)
train_images = np.array(train_images, dtype=np.float32)

In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
validate_labels = validate[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
validate_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in validate['ImagePath']]
validate_images = []
for validate_image_path in validate_image_paths:
    image = load_img(validate_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    validate_images.append(image)
validate_images = np.array(validate_images, dtype=np.float32)

In [ ]:
validate_images.shape

In [ ]:
validate_labels.shape

In [ ]:
test_labels = la2[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
test_image_paths = [os.path.join('/workspace/anushka saini/test/images', filename) for filename in la2['imagePath']]
test_images = []
for test_image_path in test_image_paths:
    image = load_img(test_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    test_images.append(image)
test_images = np.array(test_images, dtype=np.float32)

In [ ]:
test_images.shape

In [ ]:
test_labels.shape

### TL_Models

1) **ConvNeXtBase**

In [ ]:
base_model=ConvNeXtBase(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("ConvNeXtBase_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('ConvNeXtBase_weights.h5')
model.save('ConvNeXtBase_model.h5')
model.load_weights('ConvNeXtBase_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

2) **DenseNet169**

In [ ]:
base_model=DenseNet169(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("DenseNet169_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('DenseNet169_weights.h5')
model.save('DenseNet169_model.h5')
model.load_weights('DenseNet169_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

3) **InceptionResNetV2**

In [ ]:
base_model=InceptionResNetV2(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("InceptionResNetV2_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('InceptionResNetV2_weights.h5')
model.save('InceptionResNetV2_model.h5')
model.load_weights('InceptionResNetV2_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

4) **InceptionV3**

In [ ]:
base_model=InceptionV3(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("InceptionV3_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('InceptionV3_weights.h5')
model.save('InceptionV3_model.h5')
model.load_weights('InceptionV3_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

5) **MobileNetV2**

In [ ]:
base_model=MobileNetV2(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])
csv_logger = CSVLogger("MobileNetV2_model_history_log.csv", append=True) #to be changed as per model    

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('MobileNetV2_weights.h5')
model.save('MobileNetV2_model.h5')
model.load_weights('MobileNetV2_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

6) **NASNetMobile**

In [ ]:
base_model=NASNetMobile(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False    
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("NASNetMobile_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('NASNetMobile_weights.h5')
model.save('NASNetMobile_model.h5')
model.load_weights('NASNetMobile_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

7) **ResNet50V2**

In [ ]:
base_model=ResNet50V2(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])    
csv_logger = CSVLogger("ResNet50V2_model_history_log.csv", append=True) #to be changed as per model

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('ResNet50V2_weights.h5')
model.save('ResNet50V2_model.h5')
model.load_weights('ResNet50V2_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

8) **VGG19**

In [ ]:
base_model=VGG19(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])
csv_logger = CSVLogger("VGG19_model_history_log.csv", append=True) #to be changed as per model    

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('VGG19_weights.h5')
model.save('VGG19_model.h5')
model.load_weights('VGG19_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))

9) **Xception**

In [ ]:
base_model=Xception(weights='imagenet', include_top=False,input_shape=(300,300,3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])
csv_logger = CSVLogger("Xception_model_history_log.csv", append=True) #to be changed as per model    

In [ ]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

In [ ]:
model.save_weights('Xception_weights.h5')
model.save('Xception_model.h5')
model.load_weights('Xception_weights.h5')

In [ ]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

In [ ]:
print(classification_report(validate_labels,validate_labels_pred))

In [ ]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

In [ ]:
print(classification_report(test_labels,test_labels_pred))